# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,eureka,40.8021,-124.1637,292.41,77,4,1.34,US,1695494204
1,1,mejit,10.2753,170.8646,301.59,73,100,3.34,MH,1695494204
2,2,albany,42.6001,-73.9662,288.38,79,100,0.89,US,1695494194
3,3,blackmans bay,-43.0167,147.3167,280.11,93,31,2.59,AU,1695494204
4,4,utrik,11.2278,169.8474,301.57,75,100,8.81,MH,1695494204


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City")
plt.savefig("../output_data/MapCityData.png")
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

<Figure size 640x480 with 0 Axes>

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
#ideal conditions
    #A max temperature lower than 27 degrees but higher than 21
    #Wind speed less than 4.5 m/s
    #Zero cloudiness

city_data_narrow_df = city_data_df.loc[(city_data_df['Max Temp']<27)& (city_data_df['Max Temp']>21)]
city_data_narrow_df = city_data_df.loc[(city_data_df['Wind Speed']<4.5)]
city_data_narrow_df = city_data_df.loc[(city_data_df['Cloudiness']==0)]

    
# Drop any rows with null values
city_data_narrow_df = city_data_narrow_df.dropna()

# Display sample data
city_data_narrow_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,codrington,-38.2667,141.9667,280.28,81,0,3.27,AU,1695494204
21,21,aasiaat,68.7098,-52.8699,272.85,62,0,3.77,GL,1695494209
41,41,jamestown,42.0970,-79.2353,294.92,37,0,5.14,US,1695494058
42,42,sant antoni de portmany,38.9807,1.3036,295.98,63,0,2.06,ES,1695494211
51,51,lupane,-18.9315,27.8070,295.45,23,0,2.88,ZW,1695494212
...,...,...,...,...,...,...,...,...,...,...
558,558,flin flon,54.7682,-101.8650,294.06,43,0,4.63,CA,1695494298
562,562,ambovombe,-25.1667,46.0833,296.21,54,0,4.10,MG,1695494298
570,570,tolanaro,-25.0319,46.9987,296.19,78,0,11.83,MG,1695494299
578,578,craig,40.5153,-107.5464,288.12,29,0,1.54,US,1695494300


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,eureka,US,40.8021,-124.1637,77,
1,mejit,MH,10.2753,170.8646,73,
2,albany,US,42.6001,-73.9662,79,
3,blackmans bay,AU,-43.0167,147.3167,93,
4,utrik,MH,11.2278,169.8474,75,
...,...,...,...,...,...,...
581,dawson creek,CA,55.7666,-120.2362,41,
582,kincardine,GB,56.0667,-3.7167,74,
583,luwero,UG,0.8492,32.4731,88,
584,sur,OM,22.5667,59.5289,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
eureka - nearest hotel: Eureka Inn, Trademark Collection by Wyndham
mejit - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
utrik - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
mga - nearest hotel: No hotel found
tazovsky - nearest hotel: Тазовчанка
afaahiti - nearest hotel: Omati Lodge
ancud - nearest hotel: Cabañas
puerto natales - nearest hotel: Dorotea Patagonia Hostel
kuta - nearest hotel: Ida Hotel Bali
vanino - nearest hotel: Пять Звёзд
luganville - nearest hotel: Beachfront Resort
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
shangrao - nearest hotel: 心水 · 漫酒店
tanggu - nearest hotel: 汉庭酒店
grytviken - nearest hotel: No hotel found
sungai raya - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
adamstown - nearest hotel: No hotel found
aasiaat - nearest hotel: SØMA
imperatriz - nearest hotel: Hotel Posseidon
dera bugti - nearest

sakaiminato - nearest hotel: かぐら旅館
katsuren-haebaru - nearest hotel: No hotel found
trevelin - nearest hotel: Casa de Piedra
anadyr - nearest hotel: Гостевой дом
san juan de miraflores - nearest hotel: Venue Hospedaje
kirensk - nearest hotel: Ивушка
al kharj - nearest hotel: شقق فندقية
zaragoza - nearest hotel: Hotel Pilar Plaza
east ballina - nearest hotel: Ballina Beach Resort
machico - nearest hotel: Residence Inn
stanley - nearest hotel: Hotel 52
mariestad - nearest hotel: Stadshotellet
nchelenge - nearest hotel: No hotel found
fale old settlement - nearest hotel: No hotel found
ha tinh - nearest hotel: Hong Chau Nha Nhgi
elliot lake - nearest hotel: Hampton Inn Elliot Lake
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
remire-montjoly - nearest hotel: Complexe Belova
lebedyn - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
cullinan - nearest hotel: No hotel found
khanty-mansiysk - nearest hotel: Гамма
thames - nearest hotel: K

barreiras - nearest hotel: Principe Hotel
kapoeta - nearest hotel: Kuleu
broken hill - nearest hotel: Ibis Styles
laguna - nearest hotel: Holiday Inn Express & Suites
kerikeri - nearest hotel: Avalon Resort
sinop - nearest hotel: No hotel found
ilok - nearest hotel: Stari podrumi
nemuro - nearest hotel: イーストハーバーホテル
onguday - nearest hotel: Харлей
jumla - nearest hotel: Amar Sandesh Guesthouse
atafu village - nearest hotel: No hotel found
envira - nearest hotel: No hotel found
villarrobledo - nearest hotel: Hotel Juan Carlos I
rishikesh - nearest hotel: Moksha
west end - nearest hotel: The Inn at Old Bahama Bay
maun - nearest hotel: Center Lodge Conference Center
mar del plata - nearest hotel: Nuevo Ostende
mormugao - nearest hotel: Hotel Rukhmini
kayangel - nearest hotel: No hotel found
puerto aysen - nearest hotel: Hostal Dendal
mana - nearest hotel: Le Samana
dalandzadgad - nearest hotel: Хан Уул буудал
baker city - nearest hotel: Geiser Grand Hotel
al bardiyah - nearest hotel: فندق 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot

hoteldata =hotel_df.loc[hotel_df["Hotel Name"] != "N/A"]

map_plot_2= hoteldata.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=1,
    color = "City",
    hover_cols = ["City Name", "Max Temp", "Hotel Name", "Hotel Address"]

)
plt.savefig("../output_data/MapHotelData.png")

# Display the map
map_plot_2